# Testando Detectores x Descritores

In [1]:
import cv2
import numpy as np
import pandas as pd
import itertools

## Funções

In [2]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    ret, dst = cv2.threshold(dst,threshold*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image


In [3]:
def detectSIFTKeypoints(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None)
    not_dup_kp = {pt.pt: pt for pt in kp}
    kp_ = list(not_dup_kp.values())
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 255, 0), flags=0)

    return kp_, kp_image 

In [4]:
def detectStarKeypoints(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kp = star.detect(gray_image, None)    

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image

In [5]:
from sklearn.metrics import pairwise_distances_argmin_min

def computeDistacesKeypoints(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)
        matches_pts1 = [pts1[i] for i, (pt, dist) in enumerate(zip(dists[0], dists[1])) if dist <= threshold]
        matches_pts2 = [pts2[pt] for pt, dist in zip(dists[0], dists[1]) if dist <= threshold]
    else:
        matches_pts1 = []
        matches_pts2 = []
        
    return matches_pts1, matches_pts2


def return_detector_func(algol):
    if algol == 'harris':
        func = detectHarrisKeypoints
    elif algol == 'sift':
        func = detectSIFTKeypoints
    else:
        func = detectStarKeypoints
    
    return func


def compare_detectors_keypoints(image, algol_1, algol_2, params_algol_1, params_algol_2, threshold):
    detector_1 = return_detector_func(algol_1)
    detector_2 = return_detector_func(algol_2)

    kp1, _ = detector_1(image, **params_algol_1)
    kp2, _ = detector_2(image, **params_algol_2)

    kp1 = [(kp.pt[0], kp.pt[1]) for kp in kp1]
    kp2 = [(kp.pt[0], kp.pt[1]) for kp in kp2]

    matches_pts1, matches_pts2 = computeDistacesKeypoints(kp1, kp2, threshold=threshold)
    mismatches_pts1 = [pt for pt in kp1 if pt not in matches_pts1]
    mismatches_pts2 = [pt for pt in kp2 if pt not in matches_pts2]

    matches_kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in matches_pts1]
    matches_kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in matches_pts2]

    mismatches_kp1 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in mismatches_pts1]
    mismatches_kp2 = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in mismatches_pts2]

    kp_image = cv2.drawKeypoints(image, matches_kp1, None, color=(0, 255, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, matches_kp2, None, color=(0, 255, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, mismatches_kp1, None, color=(255, 0, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, mismatches_kp2, None, color=(0, 0, 255), flags=0)

    return kp_image, matches_kp1, matches_kp2, kp1, kp2 

def return_params_combination(combination, params_harris, params_sift, params_star, pos):
    if combination == 'harris':
        params = {key: value[pos] for key, value in zip(params_harris.keys(), params_harris.values())}
    elif combination == 'sift':
        params = {key: value[pos] for key, value in zip(params_sift.keys(), params_sift.values())}
    else:
        params = {key: value[pos] for key, value in zip(params_star.keys(), params_star.values())}
    
    return params


def run_all_tests(image, params_harris, params_sift, params_star, thresholds):
    n_pontos = ['25', '50', '100']
    algol_combinations = [('harris', 'sift'), ('harris', 'star'), ('sift', 'star')]
    df = pd.DataFrame(columns=['combination', 'qtd_kp1', 'qtd_kp2', 'qtd_matches_kp1',
                                'qtd_matches_kp2', 'precision_1', 'precision_2'])
    
    for comb in algol_combinations:
        for i,n in enumerate(n_pontos):
            image_ = image.copy()
            params1 = return_params_combination(comb[0], params_harris, params_sift, params_star, i)
            params2 = return_params_combination(comb[1], params_harris, params_sift, params_star, i)
            
            result = compare_detectors_keypoints(image_, comb[0], comb[1], params1, params2, thresholds[i])
            
            kp_image, matches_kp1, matches_kp2, kp1, kp2 = result


            df.loc[len(df.index)] = [comb, len(kp1), len(kp2), len(matches_kp1), len(matches_kp2), len(matches_kp1) / len(kp1),
                               len(matches_kp2) / len(kp2) ]

            path_img = f'../results/first_part_detectors/matches_{comb[0]}_{comb[1]}_{n}_pontos.jpg'
            cv2.imwrite(path_img, kp_image)

    return df

## Descritores

### Funções

In [6]:
import copy

def calculate_slope(pt1, pt2):
    slope = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0]) if (pt2[0] - pt1[0]) != 0 else -1
    return slope


def verify_slope(match, trainKeypoints, queryKeypoints, slope1, slope2):
    slope = calculate_slope(trainKeypoints[match.trainIdx].pt, queryKeypoints[match.queryIdx].pt) 
    if slope <= slope2 and slope >= slope1:
        return True
    else:
        return False


def verify_dist(match, trainKeypoints, queryKeypoints, dist1, dist2):
    point1 = np.asarray(trainKeypoints[match.trainIdx].pt)
    point2 = np.asarray(queryKeypoints[match.queryIdx].pt)

    dist = np.linalg.norm(point1 - point2)

    if dist <= dist2 and dist >= dist1:
        return True
    else:
        return False


def calculate_slope_list(matches, matchesMask, trainKeypoints, queryKeypoints):
    slope_list = [calculate_slope(trainKeypoints[matches[i][0].trainIdx].pt, queryKeypoints[matches[i][0].queryIdx].pt) for i, m in enumerate(matchesMask)]
    slope_list_mask = [calculate_slope(trainKeypoints[matches[i][0].trainIdx].pt, queryKeypoints[matches[i][0].queryIdx].pt) for i, m in enumerate(matchesMask) if m[0] > 0]

    return slope_list, slope_list_mask


def calculate_dist_list(matches, matchesMask, trainKeypoints, queryKeypoints):
    kps_train = [(trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]) for i, m in enumerate(matchesMask)]
    kps_query = [(queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]) for i, m in enumerate(matchesMask)]

    kps_train_ = [(trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]) for i, m in enumerate(matchesMask) if m[0] > 0]
    kps_query_ = [(queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]) for i, m in enumerate(matchesMask) if m[0] > 0]
    
    array_pts_train = np.asarray(kps_train)
    array_pts_query = np.asarray(kps_query)

    array_pts_train_ = np.asarray(kps_train_)
    array_pts_query_ = np.asarray(kps_query_)

    dists = [np.linalg.norm(point1 - point2) for point1, point2 in zip(array_pts_train, array_pts_query)]
    dists_mask = [np.linalg.norm(point1 - point2) for point1, point2 in zip(array_pts_train_, array_pts_query_)]

    return dists, dists_mask


def filter_matches_by_percentile(matches, matchesMask, trainKeypoints, queryKeypoints, q1, q2):
    slope_list, slope_list_ = calculate_slope_list(matches, matchesMask, trainKeypoints, queryKeypoints)
    dists, dists_ = calculate_dist_list(matches, matchesMask, trainKeypoints, queryKeypoints)

    s1_, s2_ = np.percentile(slope_list_, [q1 ,q2])
    d1_, d2_ = np.percentile(dists_, [q1 ,q2])
    
    matchesMask_ = [ [1,0] if m[0] > 0 and verify_slope(matches[i][0], trainKeypoints, queryKeypoints, s1_, s2_) else [0,0] for i, m in enumerate(matchesMask)]
    matchesMask_ = [ [1,0] if m[0] > 0 and verify_dist(matches[i][0], trainKeypoints, queryKeypoints, d1_, d2_) else [0,0] for i, m in enumerate(matchesMask_)]

    qtd_matches = len(matches)
    qtd_matches_dlowe = len([m[0] for m in matchesMask if m[0] > 0 ])
    qtd_matches_filter = len([m[0] for m in matchesMask_ if m[0] > 0 ])

    qtd_fp_dlowe = qtd_matches - qtd_matches_dlowe
    qtd_fp_filter = qtd_matches_dlowe - qtd_matches_filter

    var_dist = np.var([d for m, d in zip(matchesMask, dists) if m[0] > 0 ])
    var_slope = np.var([sl for m, sl in zip(matchesMask, slope_list) if m[0] > 0 ])

    return matchesMask_, qtd_matches, qtd_matches_dlowe, qtd_matches_filter, qtd_fp_dlowe, qtd_fp_filter, var_dist, var_slope


def filter_matches_by_variance(matches, trainKeypoints, queryKeypoints, var_dist, var_slope):
    matchesMask_ = [[0,0] for i in range(len(matches))]
    var_dist_ = -1
    var_slope_ = -1
    ratio_test_ = -1
    
    for ratio_test in np.arange(0.1, 1., 0.01):
        matchesMask = [[0,0] for i in range(len(matches))]

        for i,(m,n) in enumerate(matches):
            if m.distance < ratio_test*n.distance:
                matchesMask[i]=[1,0]

        slope_list, slope_list_ = calculate_slope_list(matches, matchesMask, trainKeypoints, queryKeypoints)
        dists, dists_ = calculate_dist_list(matches, matchesMask, trainKeypoints, queryKeypoints)

        var_d = np.var(dists_)
        var_s = np.var(slope_list_)

        if var_d <= var_dist and var_s <= var_slope:
            ratio_test_ = round(copy.copy(ratio_test), 2)
            var_dist_ = var_d
            var_slope_ = var_s
            matchesMask_ = [m for m in matchesMask]
        
    qtd_matches = len(matches)
    qtd_matches_dlowe = len([m[0] for m in matchesMask_ if m[0] > 0 ])

    qtd_fp_dlowe = qtd_matches - qtd_matches_dlowe

    return matchesMask_, ratio_test_, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe, var_dist_, var_slope_


def draw_matches(train_img, query_img, trainKeypoints, queryKeypoints, matches, matchesMask):
    kps_train = [[trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]] for i, m in enumerate(matchesMask)]
    kps_query = [[queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]] for i, m in enumerate(matchesMask)]

    # draw the tracks
    mask = np.zeros_like(train_img, 'uint8') 

    line = (0,255,0)
    point_true = (255,0,0)
    point_false = (0,0,255)
    frame = query_img.copy()
    
    for i, (m, query, train) in enumerate(zip(matchesMask, kps_query, kps_train)):
        a, b = query[0], query[1]
        c, d = train[0], train[1]
        
        a, b = int(a), int(b)
        c, d = int(c), int(d)
        
        if m[0] > 0:
            mask = cv2.line(mask, (a, b), (c, d), line, 2) 
            frame = cv2.circle(frame, (a, b), 4, point_true, -1)
        else:
            frame = cv2.circle(frame, (a, b), 4, point_false, -1)
    
    img = cv2.add(frame, mask)

    return img


In [7]:
# def matchDetectedKeypoints(train_img, query_img, var_dist, var_slope, detector='harris', descriptor='orb', dect_kargs={}, des_kargs={}):
#     query_img_bw = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY) 
#     train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

#     if descriptor == 'brief':
#         des = cv2.xfeatures2d.BriefDescriptorExtractor_create(**des_kargs)
#         norm_type = cv2.NORM_HAMMING
#     elif descriptor == 'brisk':
#         des = cv2.BRISK_create(**des_kargs)
#         norm_type = cv2.NORM_HAMMING
#     else:
#         des = cv2.SIFT.create(**des_kargs)
#         norm_type = cv2.NORM_L2
    
#     if detector == 'star':
#         func_var = detectStarKeypoints
#     elif detector == 'harris':
#         func_var = detectHarrisKeypoints
#     else:
#         func_var = detectSIFTKeypoints

#     queryKeypoints, _ = func_var(query_img, **dect_kargs)
#     trainKeypoints, _ = func_var(train_img, **dect_kargs)
    
#     _,queryDescriptors = des.compute(query_img_bw, queryKeypoints) 
#     _,trainDescriptors = des.compute(train_img_bw, trainKeypoints)


#     matcher = cv2.BFMatcher(normType=norm_type, crossCheck=False) 
#     matches = matcher.knnMatch(queryDescriptors,trainDescriptors, k=2) 

#     # Need to draw only good matches, so create a mask
#     # matchesMask = [[0,0] for i in range(len(matches))]

#     # for i,(m,n) in enumerate(matches):
#     #     if m.distance < ratio_test*n.distance:
#     #         matchesMask[i]=[1,0]

#     # m = filter_matches_by_percentile(matches, matchesMask, trainKeypoints, queryKeypoints, filter_slope[0], filter_slope[1])
    
#     m = filter_matches_by_variance(matches, trainKeypoints, queryKeypoints, var_dist, var_slope)
#     matchesMask_, ratio_test_, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe, var_dist_, var_slope_ = m 

#     draw_params = dict(matchColor = (0,255,0),
#                     singlePointColor = (255,0,0),
#                     matchesMask = matchesMask_,
#                     flags = cv2.DrawMatchesFlags_DEFAULT)
    
#     # final_img = cv2.drawMatchesKnn(query_img,queryKeypoints,train_img,trainKeypoints,matches,None,**draw_params)
#     qtd_kp_train = len(trainKeypoints)
#     qtd_kp_query = len(queryKeypoints)

#     final_img = draw_matches(train_img, query_img, trainKeypoints, queryKeypoints, matches, matchesMask_)

#     final_img = cv2.resize(final_img, (480,640))

#     return final_img, qtd_kp_train, qtd_kp_query, ratio_test_, var_dist_, var_slope_, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe 


In [84]:
from sklearn.metrics import pairwise_distances_argmin_min

def return_idx_points_at_distance(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)
        matches = [(i, pos) for i, (pos, dist) in enumerate(zip(dists[0], dists[1])) if dist <= threshold]
    else:
        matches = []
        
    return matches


def filter_matches(kpts1, kpts2, matches, threshold=1):
    kp1 = [(kp.pt[0], kp.pt[1]) for kp in kpts1]
    kp2 = [(kp.pt[0], kp.pt[1]) for kp in kpts2]
    
    matches_distance_idx = return_idx_points_at_distance(kp1, kp2, threshold)
    matchesMask_ = [ [1,0] if (m[0].trainIdx, m[0].queryIdx) in matches_distance_idx else [0,0] for m in matches]

    return matchesMask_


def compare_descriptors(keypoints1, keypoints2, descriptors1,  descriptors2, window_size, threshold):

    # Iterate through descriptors in the second image and compare with descriptors in the window
    matches = []
    for i, descriptor2 in enumerate(descriptors2):
        x, y = keypoints2[i].pt
        distances = []

        # Define the window coordinates
        window_x1 = int(x - window_size / 2)
        window_y1 = int(y - window_size / 2)
        window_x2 = int(x + window_size / 2)
        window_y2 = int(y + window_size / 2)

        # Ensure the window is within image bounds
        for j, descriptor1 in enumerate(descriptors1):
            x2, y2 = keypoints1[j].pt
            if x2 > window_x1 and y2 > window_y1  and \
                x2 <= window_x2 and y2 <= window_y2 :

                # Calculate Euclidean distance between descriptor1 and descriptors in the window
                distance = np.linalg.norm(descriptor2 - descriptor1)
                distances.append((j, distance))

        # Check if any descriptor in the window is below the threshold
        distances = sorted(distances, key=lambda x: x[1])
        print(distances)
        if len(distances) > 0 and distances[0][1] <= threshold:
            match = cv2.DMatch(_queryIdx=distances[0][0], _trainIdx=i, _distance=np.min(distances))
            matches.append((match, match))
    
    matches = tuple(tuple(m) for m in matches)
    return matches


def matchDetectedKeypoints(train_img, query_img, threshold=5, window_size=30, ratio_test=0.6, detector='harris', descriptor='sift', dect_kargs={}, des_kargs={}):
    query_img_bw = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY) 
    train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

    if descriptor == 'brief':
        des = cv2.xfeatures2d.BriefDescriptorExtractor_create(**des_kargs)
        norm_type = cv2.NORM_HAMMING
    elif descriptor == 'brisk':
        des = cv2.BRISK_create(**des_kargs)
        norm_type = cv2.NORM_HAMMING
    else:
        des = cv2.SIFT.create(**des_kargs)
        norm_type = cv2.NORM_L2
    
    if detector == 'star':
        func_var = detectStarKeypoints
    elif detector == 'harris':
        func_var = detectHarrisKeypoints
    else:
        func_var = detectSIFTKeypoints

    queryKeypoints, _ = func_var(query_img, **dect_kargs)
    trainKeypoints, _ = func_var(train_img, **dect_kargs)
    
    _,queryDescriptors = des.compute(query_img_bw, queryKeypoints) 
    _,trainDescriptors = des.compute(train_img_bw, trainKeypoints)


    # matcher = cv2.BFMatcher(normType=norm_type, crossCheck=False) 
    # matches = matcher.knnMatch(queryDescriptors, trainDescriptors, k=2)

    matches = compare_descriptors(queryKeypoints, trainKeypoints, 
                                  queryDescriptors, trainDescriptors, window_size, threshold)
    
    # Need to draw only good matches, so create a mask
    matchesMask = [[1,0] for i in range(len(matches))]

    # for i,(m,n) in enumerate(matches):
    #     if m.distance < ratio_test*n.distance:
    #         matchesMask[i]=[1,0]

    # matchesMask = filter_matches(trainKeypoints, queryKeypoints, matches, threshold=threshold)

    draw_params = dict(
                    matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    
    final_img = cv2.drawMatchesKnn(query_img, queryKeypoints, train_img, trainKeypoints, matches, None,**draw_params)
    # final_img = cv2.drawMatches(train_img, trainKeypoints, query_img, queryKeypoints, matches, None, **draw_params)
    
    qtd_kp_train = len(trainKeypoints)
    qtd_kp_query = len(queryKeypoints)

    final_img = cv2.resize(final_img, (1280,480))

    return final_img


### dsc07631.jpg

In [85]:
params_harris = {
    'threshold' : [0.27152, 0.078, 0.053],
    'blockSize' : [2, 5, 3],
    'ksize' : [3, 5, 9],
    'k' : [0.04, 0.04, 0.06]
}

params_sift = {
    'nfeatures': [34, 69, 143],
    'nOctaveLayers' : [3, 10, 10],
    'contrastThreshold' : [0.15, 0.05, 0.01],
    'edgeThreshold' : [2, 10, 3],
    'sigma' : [1.6, 3.2, 3.2],
    'enable_precise_upscale': [True, True, True]
}

params_star = {
    'max_size': [10, 16, 10],
    'response_threshold': [70, 55, 30],
    'line_threshold_projected': [10, 5, 5],
    'line_threshold_binarized': [10, 5, 5],
    'suppress_nonmax_size': [10, 16, 10]
}

thresholds = [5,5,5]

In [86]:
# query_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075107.jpg') 
# train_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg')
# query_img_1_ = cv2.resize(query_img_1, (480,640))
# train_img_1_ = cv2.resize(train_img_1, (480,640))


query_img_2 = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img_2 = cv2.imread('../data/imgs/dsc07631.jpg')
query_img_2_ = cv2.resize(query_img_2, (640,480))
train_img_2_ = cv2.resize(train_img_2, (640,480))

# query_img_3 = cv2.imread('../data/imgs/dsc02596.jpg') 
# train_img_3 = cv2.imread('../data/imgs/dsc02595.jpg')
# query_img_3_ = cv2.resize(query_img_3, (480,640))
# train_img_3_ = cv2.resize(train_img_3, (480,640))


In [94]:
dect_kargs = {k: v[0] for k, v in params_star.items()}
des_kargs = {k: v[0] for k, v in params_sift.items()}
final_image = matchDetectedKeypoints(train_img_2, query_img_2, window_size=50, threshold=1300, ratio_test=0.6, detector='star', descriptor='brisk', dect_kargs=dect_kargs, des_kargs={})
# Show the final image 
cv2.imshow("Matches", final_image) 
cv2.waitKey()

[(2, 884.9401109679683), (1, 911.7307716645304)]
[(2, 731.469069749364)]
[(3, 1269.0933771791579)]
[(3, 1288.575958180192)]
[(6, 504.8177889100185), (7, 1376.4654009454796)]
[(8, 763.4906679193924)]
[(5, 1019.6283636698226), (9, 1149.0957314340699)]
[(7, 1087.9135995105494)]
[(8, 1049.0295515379917)]
[(9, 718.4337130174224)]
[(10, 825.5089339310629)]
[(11, 679.283445992908), (10, 1246.5472313554749)]
[(13, 550.8756665528075), (12, 927.6055196041041)]
[(12, 735.6908318036864)]
[(14, 1206.1131787688914)]
[(16, 863.686864552194)]
[(15, 695.4940689898082)]
[(17, 1010.5206578788976)]
[]
[(18, 669.6349751917085)]
[]
[(21, 925.5420033688368)]
[(21, 857.2875830198406)]
[]
[]
[(24, 510.2450391723569)]


-1

In [21]:
# run_all_tests(image, params_harris, params_sift, params_star, thresholds)